In [41]:
import spacy
from pyvis.network import Network
from transformers import pipeline


/home/arunav/.conda/envs/tf-gpu/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-08 21:23:58.310938: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-08 21:23:58.335795: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-08 21:23:59.172388: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warni

In [42]:
# Load the SpaCy NLP model
nlp = spacy.load('en_core_web_sm')

# Load the Hugging Face transformers pipeline for question answering
qa_pipeline = pipeline("question-answering")

# Sample story text
text = """Once upon a time, there was a young prince named Arthur. He embarked on a quest to find a magical sword. Along the way, he met a wise old man who gave him a map. With the map, Arthur navigated through a dark forest and reached a hidden cave. Inside the cave, he found the sword guarded by a dragon. After a fierce battle, Arthur defeated the dragon and claimed the sword."""

# Process the text with SpaCy
doc = nlp(text)

# Extract sentences
sentences = [sent.text for sent in doc.sents]

# Function to extract relationships using the QA pipeline
def extract_relationships(sentences, context):
    relationships = []
    for i in range(len(sentences) - 1):
        question = f"What happens after '{sentences[i]}'?"
        result = qa_pipeline(question=question, context=context)
        answer = result['answer']
        relationships.append((sentences[i], answer, sentences[i + 1]))
    return relationships

# Extract relationships for the entire story
relationships = extract_relationships(sentences, text)

print("Relationships:", relationships)


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Relationships: [('Once upon a time, there was a young prince named Arthur.', 'He embarked on a quest to find a magical sword', 'He embarked on a quest to find a magical sword.'), ('He embarked on a quest to find a magical sword.', 'he met a wise old man who gave him a map', 'Along the way, he met a wise old man who gave him a map.'), ('Along the way, he met a wise old man who gave him a map.', 'navigated through a dark forest and reached a hidden cave', 'With the map, Arthur navigated through a dark forest and reached a hidden cave.'), ('With the map, Arthur navigated through a dark forest and reached a hidden cave.', 'he found the sword guarded by a dragon', 'Inside the cave, he found the sword guarded by a dragon.'), ('Inside the cave, he found the sword guarded by a dragon.', 'After a fierce battle, Arthur defeated the dragon', 'After a fierce battle, Arthur defeated the dragon and claimed the sword.')]


In [43]:
# Create a Pyvis Network object
net = Network(notebook=True, directed=True)

# Add nodes and edges based on relationships
for rel in relationships:
    net.add_node(rel[0], label=rel[0])
    net.add_node(rel[1], label=rel[1])
    net.add_edge(rel[0], rel[1], title=rel[1])

# Ensure the nodes are connected in a sequential manner
for i in range(len(relationships) - 1):
    net.add_edge(relationships[i][1], relationships[i + 1][0])

# Generate the visualization and save it as an HTML file
net.show("flow_chart_2.html")


story_flowchart.html
